In [ ]:
%run config

In [ ]:
#Depedencies 
from pyspark.sql.functions import * 

In [ ]:
#API call to grab all of the acitivites within a personal account
def activity_api_call(access_token):
    activites_url = "https://www.strava.com/api/v3/athlete/activities"
    header = {'Authorization': 'Bearer ' + access_token}
    param = {'per_page': 200, 'page': 1}
    activity_dataset = requests.get(activites_url, headers=header, params=param).json()
    
    return activity_dataset


In [ ]:
my_dataset = activity_api_call(access_token)

In [ ]:
#get all of the activity ids from the bigger dataset
#xtract all of the columns we want
def extract_activities(dataset):
    """Function to seperate activity_ids and create an activity dataframe. Returns a df of only the activity ids, and another df with more details about the activiity. """
    activity_ids = []
    start_date = []
    activity_name =[]
    distance = []
    moving_time = []
    elapsed_time = []
    sport_type = []
    total_elevation_gain =[]
    count = 0
    while count < len(dataset):
        activity_ids.append(dataset[count]['id'])
        start_date.append(dataset[count]['start_date'])
        activity_name.append(dataset[count]['name'])
        distance.append(dataset[count]['distance'])
        moving_time.append(dataset[count]['moving_time'])
        elapsed_time.append(dataset[count]['elapsed_time'])
        sport_type.append(dataset[count]['sport_type'])
        total_elevation_gain.append(dataset[count]['total_elevation_gain'])
        count += 1 
        
    #convert list to dataframe   
    from pyspark.sql.types import LongType
    activity_id_DF = spark.createDataFrame(activity_ids, LongType())
    activity_id_DF = activity_id_DF.withColumnRenamed('value', 'activity_id')\
                  .withColumn("ingest_file_name", lit("activity_ids")) \
                  .withColumn("ingested_at", lit(current_timestamp()))
    
    #columns names for initial DF
    #need to specify schema
    columns = ['activity_ids','start_date', 'activity_name', 'distance', 'moving_time','elapsed_time', 'sport_type'\
          ,'total_elevation_gain']
    #list of lists
    extracted_data = [activity_ids,start_date, activity_name, distance, moving_time,elapsed_time, sport_type\
          ,total_elevation_gain]

    import pandas as pd
    #create a pandas Dataframe, then convert to spark to write to storage
    pdf = pd.DataFrame.from_dict(dict(zip(columns, extracted_data)))
    activity_df = spark.createDataFrame(pdf)

    activity_df = activity_df.withColumn("ingest_file_name", lit("activity_information")) \
                             .withColumn("ingested_at", lit(current_timestamp()))

    return activity_id_DF, activity_df

In [ ]:
activity_id_DF, activity_df = extract_activities(my_dataset)

In [ ]:
#write the activity ids to storage, overwrite the previous iteration
def write_dataframe_to_storage(dataset, storage_path, option, mode ):
    """Function to write activity ids to storage. Will overwrite current delta file in storage
    Option refers to schema overwriteSchema or mergeSchema, mode being either overwrite or append"""
    dataset.write.format("delta")\
    .option(option, "true")\
    .mode(mode)\
    .save(storage_path)

In [ ]:
write_dataframe_to_storage(activity_id_DF,activity_id_path, "overwriteSchema","overwrite" )

In [ ]:
#read the activities from storage
stored_activity_ids = spark.read.format("delta").load(activity_id_path)

In [ ]:
#read in historical activity IDS to compare
#trying to compare current query vs historical, if nothing is written to storage ie first run, still need to execute and write original DF to storage
def get_historical_acitivies(storagepath, historical_df_to_write):
    """Retrieve record from file path, if nothing exists, insert df to write to storage"""
    from pyspark.sql.utils import AnalysisException
    try:
        #try to read from storage
        historical_dataframe = spark.read.format("delta").load(historical_activity_id_path)
    except: 
        try:
        #if that fails, write the current dataframe to storage
            write_dataframe_to_storage(historical_df_to_write,storagepath, "mergeSchema", "append" )
        finally:
            historical_dataframe = spark.read.format("delta").load(storagepath)

    return historical_dataframe

In [ ]:
historical_activites.show()

In [ ]:
historical_activites = get_historical_acitivies(historical_activity_id_path, activity_df)

In [ ]:
activity_df.show()

In [ ]:
#Extract activity ids from dataframes to list, to make comparison 
#activity_id_list = activity_id_DF.select('activity_ids').distinct().collect()
activity_id_list = stored_activity_ids.select('activity_id').distinct().rdd.flatMap(lambda x: x).collect()

historical_activity_id_list = historical_activites.select('activity_ids').distinct().rdd.flatMap(lambda x: x).collect()

activity_ids_not_written_to_storage = [x for x in activity_id_list if x not in historical_activity_id_list ]

new_activities = activity_df.filter(activity_df.activity_ids.isin(activity_ids_not_written_to_storage))

In [ ]:
write_dataframe_to_storage(new_activities, historical_activity_id_path,"mergeSchema", "append")

In [ ]:
full_activity_dataset = spark.read.format("delta").load(historical_activity_id_path)

In [ ]:
full_activity_dataset.show()

In [ ]:
#take ones not in storage, and grab them from activity df, and write those to storage

In [ ]:
#need to compare distinct values and then append those to storage
def compare_current_vs_historical():
    """Compares historical activity IDS already in storage, vs the most recent API query"""

In [ ]:
historical_activity_id_path
new_activity_ids = [x for x in activity_id_DF['activity_ids'] if x not in historical_activity_ids['activity_id'] ]

In [ ]:
activity_df.filter(df.acitivty_id.isin(activity_ids_not_written_to_storage))